<a href="https://colab.research.google.com/github/tobias-hoepfl/Digital-Organizations-SE/blob/main/learning_portfolio/6_recommendation_system_theory_Hoepfl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Theoretical understanding:**

Answers to selected questions on the fastAI chapter (https://www.kaggle.com/code/jhoward/collaborative-filtering-deep-dive/notebook)

<br>

**Practical understanding:**

see homework and code comments that I did in the corresponding assignment (assignment 7)


## Theory

**What problem does collaborative filtering solve?**

Recommend specific items or products to a specific user.

<br>

**How does it solve it?**

It analyzes the behaviour of other users and based on the similarities to them it tries make predictions for users who have not yet rated a specific item or product. 

This approach can be used by applying cosine similarity like was shown in the youtube-video recommended (https://www.youtube.com/watch?v=Fmtorg_dmM0&ab_channel=ritvikmath). 

The approach used in the fastAI course works with the intuition that there are hidden (latent) features that deterime what a user will most propably like.

<br>

**Why might a collaborative filtering predictive model fail to be a very useful recommendation system?**

- Sparsity of data
- I need a lot of data to be able to start (this might not be available for a lot of items)
- Some highly active users might introduce a bias
- Especially on less popular items there might be hardly any data

<br>

**What does a crosstab representation of collaborative filtering data look like?**

Like a matrix where the rows represent the users and the columns the items.

<br>

**What is a latent factor? Why is it "latent"?**

It is not directly visible in the data, but describes the underlying structure (e.g. a tendency to likes horror movie)

<br>

**What is an embedding matrix?**

Transforms the matrix described before in a look-up table for each item-user combination (denser than the sparse matrix)

<br>

**Why do we need Embedding if we could use one-hot-encoded vectors for the same thing?**

One-hot-encoding would transform every level to its own column, which we don't want. Therefor we use an embedding which can also display continuous features

<br>

**What does an embedding contain before we start training (assuming we're not using a pretrained model)?**

It contains random initializations for the factors.

<br>

**What is the use of bias in a dot product model?**

It helps us to account for the fact that some users inherently tend to give higher ratings or some items are inherently more popular.

<br>

**What is another name for weight decay?**

L2-regularization

Helps to prevent overfitting

<br>

**Write the equation for weight decay.**

loss_with_wd = loss + wd * (parameters**2).sum()

The normal loss is incremented by a "punishing factor" for the size of the parameters.

<br>

**What is the "bootstrapping problem" in collaborative filtering?**

Give recommendations to new users or for new items

<br>

**How could you deal with the bootstrapping problem for new users? For new movies?**

- Just give them the recommendation for an average user (e.g. overall popular movies)
- Use other data (demographics, etc.) if available (is called metadata)

<br>

**What kind of model should we use if we want to add metadata about users and items, or information such as date and time, to a collaborative filtering model?**

e.g. use a hybrid recommender system that combines collaborative filtering with content-based filtering or other methods